### This is a project based on Named Entity Recognition where we are going to use it for parsing resume

### Installing prerequisite

In [53]:
! pip install transformers[sentencepiece] datasets seqeval tokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=df25bd18fd7e250c2604faf7ce22be41515280e3ddb8434d8f5760174ebafefd
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
# upgrade and reinstall transformer accelerate
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.3 MB/s eta 0:00:00
Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 13.7 MB/s eta 0:00:00
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [54]:
from transformers import AutoModelForTokenClassification,DataCollatorForTokenClassification,BertTokenizerFast
import datasets
from datasets import load_dataset
import torch

#### Setting up device for the pytorch

In [55]:
device='cuda' if torch.cuda.is_available() ==True else 'cpu'
device

'cpu'

In [56]:
resume_parser_model=AutoModelForTokenClassification.from_pretrained('bert-base-uncased',num_labels=11)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
tokenizer=BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [58]:
# we can also use AutoTokenizer
from transformers import AutoTokenizer
auto_tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [59]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [60]:
auto_tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

### Loading our resume dataset

In [61]:
! unzip "/content/drive/MyDrive/Resume Dataset/archive.zip" -d "/content/resume dataset"

Archive:  /content/drive/MyDrive/Resume Dataset/archive.zip
  inflating: /content/resume dataset/Entity Recognition in Resumes.json  


In [130]:
import json

In [131]:
# we are loading the file once at a time because it was giving us error when loaded all at once
resume_dataset=[]
with open('/content/resume dataset/Entity Recognition in Resumes.json','r') as file:
  for line in file:
    resume_dataset.append(json.loads(line))

In [132]:
# Total no of resume in our dataset
len(resume_dataset)

220

In [133]:
import pandas as pd

In [134]:
resume_df=pd.DataFrame(resume_dataset)

In [135]:
resume_df.columns

Index(['content', 'annotation', 'extras'], dtype='object')

In [136]:
resume_df.head(5)

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",None
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",None
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",None
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",None
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",None


In [137]:
resume_df['annotation'][0]

[{'label': ['Skills'],
  'points': [{'start': 1295,
    'end': 1621,
    'text': '\n• Programming language: C, C++, Java\n• Oracle PeopleSoft\n• Internet Of Things\n• Machine Learning\n• Database Management System\n• Computer Networks\n• Operating System worked on: Linux, Windows, Mac\n\nNon - Technical Skills\n\n• Honest and Hard-Working\n• Tolerant and Flexible to Different Situations\n• Polite and Calm\n• Team-Player'}]},
 {'label': ['Skills'],
  'points': [{'start': 993,
    'end': 1153,
    'text': 'C (Less than 1 year), Database (Less than 1 year), Database Management (Less than 1 year),\nDatabase Management System (Less than 1 year), Java (Less than 1 year)'}]},
 {'label': ['College Name'],
  'points': [{'start': 939, 'end': 956, 'text': 'Kendriya Vidyalaya'}]},
 {'label': ['College Name'],
  'points': [{'start': 883, 'end': 904, 'text': 'Woodbine modern school'}]},
 {'label': ['Graduation Year'],
  'points': [{'start': 856, 'end': 860, 'text': '2017\n'}]},
 {'label': ['College 

# The labels are devided into following categories
1. Name
2. College Name
3. Degree
4. Graduation Year
5. Years of Experience
6. Companies worked at
7. Designation
8. Skills
9. Location
10. Email Address

In [138]:
resume_df.drop('extras',axis='columns',inplace=True)

In [139]:
resume_df['content'][0]

"Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya Vidyalaya\n\

### We are trying to create dataframe out of annotations column that will
* Token List
* Label List
### Below is the preprocessing function that gets token list and label list as lists from annotation column

In [140]:
token_list=[]
label_list=[]
for i in resume_df['annotation']:
  token=[]
  label=[]
  for j in i:
    token.append(j['points'][0]['text'])
    label.append(j['label'])


  token_list.append(token)
  label_list.append(label)



In [141]:
dictionary={'token_list':token_list,
            'label_list':label_list}
f_data_df=pd.DataFrame(dictionary)

### This is our in hand dataframe now for token list and label list

In [142]:
f_data_df.head()

,token_list,label_list
0,"[\n• Programming language: C, C++, Java\n• Ora...","[[Skills], [Skills], [College Name], [College ..."
1,"[indeed.com/r/Afreen-Jamadar/8baf379b705e37c6,...","[[Email Address], [Skills], [Graduation Year],..."
2,"[Teradata, Mainframe, Teradata, Mainframe, Ter...","[[Skills], [Skills], [Skills], [Skills], [Skil..."
3,[❖ Operating Environment: […] Windows95/98/XP/...,"[[Skills], [Skills], [Graduation Year], [Colle..."
4,"[MCA, EARCH ENGINE MARKETING (2 years), SEM (2...","[[Degree], [Skills], [Location], [College Name..."


In [143]:
# Flattening the label_list
for n,i in enumerate(f_data_df['label_list']):
  flat_list=sum(i,[])
  f_data_df['label_list'][n]=flat_list

In [144]:
f_data_df.head()

,token_list,label_list
0,"[\n• Programming language: C, C++, Java\n• Ora...","[Skills, Skills, College Name, College Name, G..."
1,"[indeed.com/r/Afreen-Jamadar/8baf379b705e37c6,...","[Email Address, Skills, Graduation Year, Colle..."
2,"[Teradata, Mainframe, Teradata, Mainframe, Ter...","[Skills, Skills, Skills, Skills, Skills, Skill..."
3,[❖ Operating Environment: […] Windows95/98/XP/...,"[Skills, Skills, Graduation Year, College Name..."
4,"[MCA, EARCH ENGINE MARKETING (2 years), SEM (2...","[Degree, Skills, Location, College Name, Degre..."


### Checking all the Unique Labels for Disparities

In [145]:
unique_labels=[]
for i in f_data_df['label_list']:
  for j in i:
    if j not in unique_labels:
      unique_labels.append(j)
unique_labels

['Skills',
 'College Name',
 'Graduation Year',
 'Designation',
 'Companies worked at',
 'Email Address',
 'Location',
 'Name',
 'Degree',
 'Years of Experience',
 'UNKNOWN']

### Let's find how many times unknown label is there

In [146]:
unknown_count=[]
for i in f_data_df['label_list']:
  for j in i:
    if j=='UNKNOWN':
      unknown_count.append(j)
len(unknown_count)

2

#### Index of UNKNOWN label

In [147]:
unknown_index=[]
for n,i in enumerate(f_data_df['label_list']):
  for j in i:
    if j=='UNKNOWN':
      unknown_index.append(n)
unknown_index

[61, 175]

#### Let's check out the index

In [148]:
f_data_df.iloc[61]['label_list'][8]

'UNKNOWN'

In [149]:
f_data_df.iloc[61]['token_list'][8] # so this unknown is supposed to be a skill

'Oracle 10g, 11g, 12c'

In [150]:
f_data_df.iloc[175]['label_list'][1]

'UNKNOWN'

In [151]:
f_data_df.iloc[175]['token_list'][1] # this unknown is supposed to be a graduation year

'2012\n'

In [152]:
f_data_df.iloc[175]['token_list'][0:3]

['40 WPM (Less than 1 year), AUTOCAD (Less than 1 year), EXCEL (Less than 1 year), MS EXCEL\n(Less than 1 year), MS OFFICE (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nSOFTWARE SKILLS\n• Completed successfully MS-CIT MS Office - Basic Computer Course (MS Word, MS Excel, MS\nPower Point & Internet) AutoCAD in Electrical\n\n\n\n• Good English, Marathi, & Hindi Typing Skills (30-40 Wpm)',
 '2012\n',
 'Nagpur Polytechnic -  Nagpur, Maharashtra\n']

### replacing the unknown with actual label

In [153]:
f_data_df.iloc[61]['label_list'][8]='Skills'
f_data_df.iloc[175]['label_list'][1]='Graduation Year'

In [154]:
unknown_count=[]
for i in f_data_df['label_list']:
  for j in i:
    if j=='UNKNOWN':
      unknown_count.append(j)
len(unknown_count) # here you can see we have successfully fixed the unknown

0

### Now we will split each label according to the words no of words it's sentence contains
* But first we have to remove all /n from the corpus in the token_list

In [155]:
f_data_df.head()

,token_list,label_list
0,"[\n• Programming language: C, C++, Java\n• Ora...","[Skills, Skills, College Name, College Name, G..."
1,"[indeed.com/r/Afreen-Jamadar/8baf379b705e37c6,...","[Email Address, Skills, Graduation Year, Colle..."
2,"[Teradata, Mainframe, Teradata, Mainframe, Ter...","[Skills, Skills, Skills, Skills, Skills, Skill..."
3,[❖ Operating Environment: […] Windows95/98/XP/...,"[Skills, Skills, Graduation Year, College Name..."
4,"[MCA, EARCH ENGINE MARKETING (2 years), SEM (2...","[Degree, Skills, Location, College Name, Degre..."


In [156]:
f_data_df['token_list'][3][3]

'Indira Gandhi Institute Of Technology'

### we will create a new dataframe that have words splitting and labels

In [171]:
perfect_df=pd.DataFrame(columns=['token_list','label_list'],index=f_data_df.index)
perfect_df

,token_list,label_list
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
215,NaN,NaN
216,NaN,NaN
217,NaN,NaN
218,NaN,NaN


In [172]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
all_stop_words=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### This is the main preprocessing function below, is it's detailed breakdown
* It goes through token list and label list cleans token_list from symbols and non alpha numeric characters,\n etc lowers it
* Then removes stop words from it
* Then basically tokenize to see bert tokens and take no of words from there , -2 for sos and eos token
* Then it basically puts one label to no_of_words times in some_labels so that for every token there is respective label
* Later appends every thing into all_token and all_label and that all_token and all_label goes into dataframe we created above becoming perfect df

In [175]:
for n,(i,j) in enumerate(zip(f_data_df['token_list'],f_data_df['label_list'])):
  all_token=[]
  all_label=[]
  for k,l in zip(i,j):
    some_string=k
    import regex as re
    some_string1=re.sub(r'[^a-zA-Z0-9\s]+', '',some_string).replace('\n','').lower().split(' ')
    while("" in some_string1):
        some_string1.remove("")
    some_string2=[word for word in some_string1 if word not in all_stop_words]
    ot=tokenizer(some_string2,max_length=510,truncation=True,is_split_into_words=True)
    all_token.append(some_string2)
    no_of_words=len(ot['input_ids'])-2
    some_label=l
    some_labels=[]
    for i in range(0,no_of_words):
      some_labels.append(some_label)
    all_label.append(some_labels)
  perfect_df['token_list'][n]=all_token
  perfect_df['label_list'][n]=all_label

In [176]:
len(perfect_df['token_list'][3][3])

4

In [177]:
len(perfect_df['label_list'][3][3])

4

In [178]:
perfect_df.head()

,token_list,label_list
0,"[[programming, language, c, c, java, oracle, p...","[[Skills, Skills, Skills, Skills, Skills, Skil..."
1,"[[indeedcomrafreenjamadar8baf379b705e37c6], [d...","[[Email Address, Email Address, Email Address,..."
2,"[[teradata], [mainframe], [teradata], [mainfra...","[[Skills, Skills, Skills], [Skills, Skills], [..."
3,"[[operating, environment, windows9598xpnt, dat...","[[Skills, Skills, Skills, Skills, Skills, Skil..."
4,"[[mca], [earch, engine, marketing, 2, years, s...","[[Degree], [Skills, Skills, Skills, Skills, Sk..."


In [179]:
perfect_df

,token_list,label_list
0,"[[programming, language, c, c, java, oracle, p...","[[Skills, Skills, Skills, Skills, Skills, Skil..."
1,"[[indeedcomrafreenjamadar8baf379b705e37c6], [d...","[[Email Address, Email Address, Email Address,..."
2,"[[teradata], [mainframe], [teradata], [mainfra...","[[Skills, Skills, Skills], [Skills, Skills], [..."
3,"[[operating, environment, windows9598xpnt, dat...","[[Skills, Skills, Skills, Skills, Skills, Skil..."
4,"[[mca], [earch, engine, marketing, 2, years, s...","[[Degree], [Skills, Skills, Skills, Skills, Sk..."
...,...,...
215,"[[government, engineering, college, bhuj], [en...","[[College Name, College Name, College Name, Co..."
216,"[[delhi], [delhi], [delhi], [delhi], [delhi], ...","[[Location], [Location], [Location], [Location..."
217,"[[auditing, less, 1, year, cfa, less, 1, year,...","[[Skills, Skills, Skills, Skills, Skills, Skil..."
218,"[[excel, 10, years, operations, 7, years, proj...","[[Skills, Skills, Skills, Skills, Skills, Skil..."


In [180]:
perfect_df['token_list'][3][3]

['indira', 'gandhi', 'institute', 'technology']

In [181]:
perfect_df['label_list'][3][3]

['College Name', 'College Name', 'College Name', 'College Name']

### Now time to flatten the lists withing the perfect_df

In [182]:
for n,i in enumerate(perfect_df['label_list']):
  flat_list=sum(i,[])
  perfect_df['label_list'][n]=flat_list

for n,i in enumerate(perfect_df['token_list']):
  flat_list=sum(i,[])
  perfect_df['token_list'][n]=flat_list

In [183]:
perfect_df.head()

,token_list,label_list
0,"[programming, language, c, c, java, oracle, pe...","[Skills, Skills, Skills, Skills, Skills, Skill..."
1,"[indeedcomrafreenjamadar8baf379b705e37c6, data...","[Email Address, Email Address, Email Address, ..."
2,"[teradata, mainframe, teradata, mainframe, ter...","[Skills, Skills, Skills, Skills, Skills, Skill..."
3,"[operating, environment, windows9598xpnt, data...","[Skills, Skills, Skills, Skills, Skills, Skill..."
4,"[mca, earch, engine, marketing, 2, years, sem,...","[Degree, Skills, Skills, Skills, Skills, Skill..."


In [184]:
perfect_df

,token_list,label_list
0,"[programming, language, c, c, java, oracle, pe...","[Skills, Skills, Skills, Skills, Skills, Skill..."
1,"[indeedcomrafreenjamadar8baf379b705e37c6, data...","[Email Address, Email Address, Email Address, ..."
2,"[teradata, mainframe, teradata, mainframe, ter...","[Skills, Skills, Skills, Skills, Skills, Skill..."
3,"[operating, environment, windows9598xpnt, data...","[Skills, Skills, Skills, Skills, Skills, Skill..."
4,"[mca, earch, engine, marketing, 2, years, sem,...","[Degree, Skills, Skills, Skills, Skills, Skill..."
...,...,...
215,"[government, engineering, college, bhuj, envir...","[College Name, College Name, College Name, Col..."
216,"[delhi, delhi, delhi, delhi, delhi, data, back...","[Location, Location, Location, Location, Locat..."
217,"[auditing, less, 1, year, cfa, less, 1, year, ...","[Skills, Skills, Skills, Skills, Skills, Skill..."
218,"[excel, 10, years, operations, 7, years, proje...","[Skills, Skills, Skills, Skills, Skills, Skill..."


In [185]:
input_ids=[]
token_type_ids=[]
attention_mask=[]
for n,i in enumerate(perfect_df['token_list']):
  input_encoding=tokenizer(i,is_split_into_words=True,max_length=510,truncation=True)
  input_ids.append(input_encoding['input_ids'])
  token_type_ids.append(input_encoding['token_type_ids'])
  attention_mask.append(input_encoding['attention_mask'])



perfect_df['input_ids']=input_ids
perfect_df['token_type_ids']=token_type_ids
perfect_df['attention_mask']=attention_mask

In [186]:
perfect_df.head()

,token_list,label_list,input_ids,token_type_ids,attention_mask
0,"[programming, language, c, c, java, oracle, pe...","[Skills, Skills, Skills, Skills, Skills, Skill...","[101, 4730, 2653, 1039, 1039, 9262, 14721, 724...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[indeedcomrafreenjamadar8baf379b705e37c6, data...","[Email Address, Email Address, Email Address, ...","[101, 5262, 9006, 27528, 28029, 3900, 23574, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[teradata, mainframe, teradata, mainframe, ter...","[Skills, Skills, Skills, Skills, Skills, Skill...","[101, 28774, 8447, 2696, 2364, 15643, 28774, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[operating, environment, windows9598xpnt, data...","[Skills, Skills, Skills, Skills, Skills, Skill...","[101, 4082, 4044, 3645, 2683, 28154, 2620, 259...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[mca, earch, engine, marketing, 2, years, sem,...","[Degree, Skills, Skills, Skills, Skills, Skill...","[101, 22432, 4540, 2818, 3194, 5821, 1016, 208...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


### Now we are encoding the labels

In [187]:
label_list={'Name':1,
'College Name':2,
'Degree':3,
'Graduation Year':4,
'Years of Experience':5,
'Companies worked at':6,
'Designation':7,'Skills':8,
'Location':9,
'Email Address':10}
label_list

{'Name': 1,
 'College Name': 2,
 'Degree': 3,
 'Graduation Year': 4,
 'Years of Experience': 5,
 'Companies worked at': 6,
 'Designation': 7,
 'Skills': 8,
 'Location': 9,
 'Email Address': 10}

In [188]:
for i in perfect_df['label_list']:
  for n,j in enumerate(i):
    encoded_value=label_list[j]
    i[n]=encoded_value

In [189]:
perfect_df.head()

,token_list,label_list,input_ids,token_type_ids,attention_mask
0,"[programming, language, c, c, java, oracle, pe...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[101, 4730, 2653, 1039, 1039, 9262, 14721, 724...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[indeedcomrafreenjamadar8baf379b705e37c6, data...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[101, 5262, 9006, 27528, 28029, 3900, 23574, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[teradata, mainframe, teradata, mainframe, ter...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[101, 28774, 8447, 2696, 2364, 15643, 28774, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[operating, environment, windows9598xpnt, data...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[101, 4082, 4044, 3645, 2683, 28154, 2620, 259...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[mca, earch, engine, marketing, 2, years, sem,...","[3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[101, 22432, 4540, 2818, 3194, 5821, 1016, 208...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [195]:
len(perfect_df)

220

#### Now you can see that input_ids are always ahead by 2 in length compared to our label's length this is because of sos and eos token that is extra in input_ids inorder to balance it out we will add -100 in the beginning and ending of our label_list because pytorch doesn't consider -100 in calculation so it won't be interfaring much in training

In [196]:
len(perfect_df['label_list'][12])

63

In [197]:
len(perfect_df['input_ids'][12])

65

In [198]:
for i in perfect_df['label_list']:
  i.insert(0,-100)
  i.append(-100)


perfect_df.head()

,token_list,label_list,input_ids,token_type_ids,attention_mask
0,"[programming, language, c, c, java, oracle, pe...","[-100, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[101, 4730, 2653, 1039, 1039, 9262, 14721, 724...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[indeedcomrafreenjamadar8baf379b705e37c6, data...","[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,...","[101, 5262, 9006, 27528, 28029, 3900, 23574, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[teradata, mainframe, teradata, mainframe, ter...","[-100, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[101, 28774, 8447, 2696, 2364, 15643, 28774, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[operating, environment, windows9598xpnt, data...","[-100, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[101, 4082, 4044, 3645, 2683, 28154, 2620, 259...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[mca, earch, engine, marketing, 2, years, sem,...","[-100, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[101, 22432, 4540, 2818, 3194, 5821, 1016, 208...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [199]:
perfect_df.rename(columns={'label_list':'labels'},inplace=True)

In [200]:
perfect_df.rename(columns={'token_list':'tokens'},inplace=True)

In [201]:
perfect_df.head(1)

,tokens,labels,input_ids,token_type_ids,attention_mask
0,"[programming, language, c, c, java, oracle, pe...","[-100, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[101, 4730, 2653, 1039, 1039, 9262, 14721, 724...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


### Rechecking whether inputids and labels are of same size or not

In [202]:
len(perfect_df['labels'][12])

65

In [203]:
len(perfect_df['input_ids'][12])

65

### Creating train and test splits

In [204]:
train_df=perfect_df[:210]
test_df=perfect_df[210:]

In [205]:
train_dataset=datasets.Dataset.from_dict(train_df.to_dict('list'))
test_dataset=datasets.Dataset.from_dict(test_df.to_dict('list'))

In [206]:
train_dataset

Dataset({
    features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 210
})

In [207]:
test_dataset

Dataset({
    features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10
})

In [208]:
from transformers import TrainingArguments,Trainer

In [209]:
data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer) #setting up the collator

In [ ]:
train_args=TrainingArguments(output_dir='Ner model',
                             evaluation_strategy='epoch',
                             save_strategy='epoch',
                             metric_for_best_model='loss',
                             learning_rate=2e-5,
                             per_device_train_batch_size=1,
                             per_device_eval_batch_size=1,
                             num_train_epochs=5,
                             weight_decay=0.01,
                             load_best_model_at_end=True)

In [ ]:
trainer=Trainer(model=resume_parser_model,args=train_args,data_collator=data_collator,train_dataset=train_dataset,
                eval_dataset=test_dataset,tokenizer=tokenizer)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.355163
2,No log,0.447148
3,0.129800,0.489709
4,0.129800,0.459924
5,0.073100,0.452455


TrainOutput(global_step=1000, training_loss=0.10149570846557618, metrics={'train_runtime': 115.8469, 'train_samples_per_second': 8.632, 'train_steps_per_second': 8.632, 'total_flos': 53381350600860.0, 'train_loss': 0.10149570846557618, 'epoch': 5.0})

In [ ]:
resume_parser_model.save_pretrained('resume_ner_model_saved')
tokenizer.save_pretrained('Bert_tokenizer')

('Bert_tokenizer/tokenizer_config.json',
 'Bert_tokenizer/special_tokens_map.json',
 'Bert_tokenizer/vocab.txt',
 'Bert_tokenizer/added_tokens.json',
 'Bert_tokenizer/tokenizer.json')

In [ ]:
label_list

{'Name': 1,
 'College Name': 2,
 'Degree': 3,
 'Graduation Year': 4,
 'Years of Experience': 5,
 'Companies worked at': 6,
 'Designation': 7,
 'Skills': 8,
 'Location': 9,
 'Email Address': 10}

In [ ]:
label_list['None']=11
label_list

{'Name': 1,
 'College Name': 2,
 'Degree': 3,
 'Graduation Year': 4,
 'Years of Experience': 5,
 'Companies worked at': 6,
 'Designation': 7,
 'Skills': 8,
 'Location': 9,
 'Email Address': 10,
 'None': 11}

In [ ]:
id2label={values: keys for values,keys in zip(list(label_list.values()),list(label_list.keys()))}
label2id={f'{keys}': values for values,keys in zip(list(label_list.values()),list(label_list.keys()))}

In [ ]:
id2label

{1: 'Name',
 2: 'College Name',
 3: 'Degree',
 4: 'Graduation Year',
 5: 'Years of Experience',
 6: 'Companies worked at',
 7: 'Designation',
 8: 'Skills',
 9: 'Location',
 10: 'Email Address',
 11: 'None'}

In [ ]:
label2id

{'Name': 1,
 'College Name': 2,
 'Degree': 3,
 'Graduation Year': 4,
 'Years of Experience': 5,
 'Companies worked at': 6,
 'Designation': 7,
 'Skills': 8,
 'Location': 9,
 'Email Address': 10,
 'None': 11}

In [ ]:
config = json.load(open("resume_ner_model_saved/config.json","r"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("resume_ner_model_saved/config.json","w"))

In [12]:
!unzip '/content/drive/MyDrive/Resume Dataset/Bert_tokenizer-20240525T130632Z-001.zip'
!unzip '/content/drive/MyDrive/Resume Dataset/resume_ner_model_saved-20240525T130636Z-001.zip'

Archive:  /content/drive/MyDrive/Resume Dataset/Bert_tokenizer-20240525T130632Z-001.zip
  inflating: Bert_tokenizer/vocab.txt  
  inflating: Bert_tokenizer/tokenizer.json  
  inflating: Bert_tokenizer/tokenizer_config.json  
  inflating: Bert_tokenizer/special_tokens_map.json  
Archive:  /content/drive/MyDrive/Resume Dataset/resume_ner_model_saved-20240525T130636Z-001.zip
  inflating: resume_ner_model_saved/config.json  
  inflating: resume_ner_model_saved/model.safetensors  


In [13]:
from transformers import pipeline

In [14]:
pipe=pipeline(task='ner',model='/content/resume_ner_model_saved',tokenizer='/content/Bert_tokenizer')

### Some Examples I have created

In [46]:
sent=''' Rudra Singh ,
contact rudrapratapsingh9@gmail.com, No. 7705895112,
good at python , pandas, numpy, matplotlib , scikitlearn , tensorflow , pytorch , powerbi , sql ,  looking for a role in Data science,
completed my Bachelor's in business administration from Bundelkhand university in Jhansi, previously worked at Tata consultancy as a senior Data Scientist for 5 years in Delhi,
Junior Ml Engineer in cisco for 2 in years Hyderabad,
 7 years of experience  total in predictive modeling and data analysis.
 have a look at my projects here https://github.com/rudrasingh.
'''

In [48]:
sent2='''Jahnvi singh ,
graduated from delhi university bcom hons,
3 years of experience as a data analyst ,
excellent proficiency in python , c++, java,
predictive modelling, data visualization,
you can mail me at jabs63@gmail.com,
previously worked at google as Senior data analyst for 2 years,
junior data analyst at cognizant 1 year.
reach out to me on instagram at https://instagram.com/jahnvisingh/
Number: 8498710553'''

In [233]:
sent3='''Abhishek Kumar,
Bsc in computer science from srcc delhi university,
 Proficient in Communication , highly cooperative and collaborative,Keen observer and Loves to finish projects with a touch of perfection.
done internship in Google as a Management Associate for 1 year in Delhi,
Then in Infosys worked for 2 years as a Finance Director in Pune ,
Reach out to me at Itsabhi@gmail.com
linkedn profile https://www.linkedin.com/in/abhishek-kumar-665266237/  ,
No. 9876543210 . '''

In [ ]:
predictions=pipe(sent)

In [ ]:
predictions

[{'entity': 'Name',
  'score': 0.23827407,
  'index': 1,
  'word': 'hi',
  'start': 0,
  'end': 2},
 {'entity': 'Skills',
  'score': 0.18938448,
  'index': 2,
  'word': 'my',
  'start': 3,
  'end': 5},
 {'entity': 'Name',
  'score': 0.3900047,
  'index': 3,
  'word': 'name',
  'start': 6,
  'end': 10},
 {'entity': 'Name',
  'score': 0.34994105,
  'index': 4,
  'word': 'is',
  'start': 11,
  'end': 13},
 {'entity': 'Name',
  'score': 0.94775546,
  'index': 5,
  'word': 'yu',
  'start': 14,
  'end': 16},
 {'entity': 'Name',
  'score': 0.94746643,
  'index': 6,
  'word': '##g',
  'start': 16,
  'end': 17},
 {'entity': 'Name',
  'score': 0.9669744,
  'index': 7,
  'word': 'pr',
  'start': 18,
  'end': 20},
 {'entity': 'Name',
  'score': 0.95643854,
  'index': 8,
  'word': '##ata',
  'start': 20,
  'end': 23},
 {'entity': 'Name',
  'score': 0.9528726,
  'index': 9,
  'word': '##p',
  'start': 23,
  'end': 24},
 {'entity': 'Name',
  'score': 0.890155,
  'index': 10,
  'word': 'singh',
  'sta

# Creating the flask app

In [15]:
! unzip '/content/drive/MyDrive/Resume Dataset/static.zip'
!unzip '/content/drive/MyDrive/Resume Dataset/templates.zip'
# remember to rename files in case of error

Archive:  /content/drive/MyDrive/Resume Dataset/static.zip
   creating: static/
   creating: static/.ipynb_checkpoints/
   creating: static/css/
   creating: static/images/
   creating: static/css/.ipynb_checkpoints/
  inflating: static/css/style.css    
  inflating: static/css/main.css     
  inflating: static/images/wp7447287-cv-wallpapers.jpg  
Archive:  /content/drive/MyDrive/Resume Dataset/templates.zip
   creating: templates/
   creating: templates/.ipynb_checkpoints/
  inflating: templates/result.html   
  inflating: templates/base.html     
  inflating: templates/index.html    


In [16]:
from flask import Flask,render_template,request,jsonify
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
!pip install pyngrok

In [18]:
from pyngrok import ngrok

In [19]:
port_no=5000

In [235]:
from flask import Flask, render_template, request
from nltk.corpus import stopwords
import re

app = Flask(__name__)
port_no = 5000  # Define the port number

ngrok.set_auth_token('2dW7qeQXYf1RbmtHWu1mUriKQnC_6qjCtetiLtbfKXVY6R1ZZ')
public_url = ngrok.connect(port_no).public_url

@app.route('/', methods=['GET'])
def homepage():
    return render_template('index.html')

@app.route('/review', methods=['POST'])
def main_function():
    if request.method == 'POST':
        try:
            resume = str(request.form.get('content'))

            email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[A-Za-z]{2,64}'
            extracted_emails = re.findall(email_pattern, resume)
            resume=resume.replace(extracted_emails[0],'')



            url_pattern =  r'(https?://(?:www\.)?[a-zA-Z0-9.-]+(?:\.[a-zA-Z]{2,})+/?[^\s]*)'
            extracted_links = re.findall(url_pattern, resume)
            for i in extracted_links:
              resume=resume.replace(i,'')


            phone_pattern = r"(?:(?:\+91)?[6789]\d{9})"
            phone_numbers=re.findall(phone_pattern, resume)
            resume=resume.replace(phone_numbers[0],'')



            predictions =pipe(resume)

            all_stop_words = stopwords.words('english')
            punctuations=[',','.','-','/']


            for prediction in predictions:

              if prediction['word'] in all_stop_words or prediction['word'] in punctuations or prediction['score'] <= 0.35 or prediction['entity'] == 'Email Address':
                    prediction['entity'] = 'Irrelevant Info'
              else:
                pass


            indexes=[]
            for i in range(0,len(predictions)):
              sub=predictions[i]
              if sub['entity']!='Irrelevant Info':
                indexes.append(i)
            filtered_results=[]
            for i in indexes:
              filtered_results.append(predictions[i])



            hash_tag_indexes=[]
            for i in range(0,len(filtered_results)):
              if filtered_results[i]['word'][0]=='#':
                j=i
                while filtered_results[j]['word'][0]=='#':
                  j=j-1

                filtered_results[j]['word']=filtered_results[j]['word']+filtered_results[i]['word'][2:]
              else:
                hash_tag_indexes.append(i)


            final_filtered_results=[]
            for i in hash_tag_indexes:
              final_filtered_results.append(filtered_results[i])


            parsed_resume = []
            parsed_resume.append({'word': extracted_emails[0], 'entity': 'Email Address' })
            parsed_resume.append({'word': phone_numbers[0], 'entity': 'Phone Number' })

            for i in extracted_links:
              parsed_resume.append({'word': i, 'entity': 'Profile Url' })

            for prediction in final_filtered_results:
              word = prediction['word']
              entity = prediction['entity']
              if entity=='Irrelevant Info':
                pass
              else:
                parsed_resume.append({'word': word, 'entity': entity})




            return render_template('result.html', parsed_resume=parsed_resume)
        except Exception as e:
            return str(e)

if __name__ == '__main__':
    print(f'To access the global link please click {public_url}')
    app.run(port=port_no)

To access the global link please click https://441b-35-197-82-229.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/May/2024 00:57:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 00:57:14] "GET /static/css/main.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 00:57:14] "GET /static/images/wp7447287-cv-wallpapers.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 00:57:14] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 00:57:21] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 00:57:23] "POST /review HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 00:57:23] "GET /static/css/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 00:58:02] "POST /review HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 00:58:02] "GET /static/css/style.css H